In [ ]:
#Initial imports
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.interpolate import interp1d

# %matplotlib notebook for interactive figures
# %matplotlib inline for bare figures
%matplotlib inline 
plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['font.size'] = 18


Self similar density profile
-----------

Self similar collapse in EdS from F&G -- $\lambda(\tau)$ is a rescaled shell orbit and for $\tau>1$ it models the motion after turnaround. Self similarity means that all of the shells follow the same eom.

\begin{equation}
    \Lambda(\tau) = \tau^{2/3+2s/9}
\end{equation}

\begin{equation}
    \frac{d^2\lambda}{d\tau^2} = - \frac{\pi}{8} \frac{\tau^{2s/3}}{\lambda^2} \mathcal{M}\left( \frac{\lambda}{\Lambda(\tau)}\right)
\end{equation}

\begin{equation}
    \mathcal{M}(y) = \frac{2s}{3} \int_1^{\infty} \frac{dx}{x^{1+2s/3}} H \left[y - \frac{\lambda(x)}{\Lambda(x)}\right]
\end{equation}

$\mathcal{M}$ is a rescaled mass profile. Notice that the last two equations involve both $\lambda(\tau)$ and $\mathcal{M}(y)$, and must be solved consistently.

Following F&G, the code below finds the solution for this system of equations by iterating the above equations a few times, starting from an initial guess for $\mathcal{M}(y)$. The code then plots $\lambda(\tau)$ for the different iterations, this result should be checked for convergence.

Please note that in this notebook $\lambda = r/r_\ast$, which is a different definition from the usual one, $\lambda = r/R$.

Variables / integration code
------

In [ ]:
s = 1.5 # Accretion rate, s>1, s>10 has very little meaning. 
tau_max = 300 # Maximum tau when computing orbits. Suggested values: 300 for s=1.5, 30 for s=5
tau_N_steps = 2e5 # Time steps. Suggested values: 1e4 for s=5, 2e5 for s=1.5

N_iterations = 10 # Number of iterations of lambda-M
N_M = 1000 #Number of grid points for M(x)

save_path = "cache/" # Saves converged lambda orbits in here

In [ ]:
#Initial guess is M(x) = x
M_interp = interp1d(np.linspace(0, 1, 100), np.linspace(0, 1, 100), fill_value='extrapolate')

#tau steps
ts = np.linspace(1, tau_max, int(tau_N_steps))
dt = ts[1]-ts[0]

#**STIFF**
big_number = 1e20
width = 1e-5

# grid for M
xs = np.linspace(0., 1., N_M)


#M function
def M(x):
    return M_interp(x)

# Lambda function
def Lambda(t):
    return t**(2./3. + 2.*s/9.)

# Derivative of Lambda
def Lambda_p(t):
    return (2./3. + 2.*s/9.)* t**(2./3. + 2.*s/9.-1.)

#Hamiltonian
def f(X, t):
    # X[0] is lambda, X[1] is lambda'
    dx0 = X[1]
    dx1 = -np.pi*np.pi/8. * t**(2.*s/3.)/(X[0]**2.) * M(X[0]/Lambda(t)) 
    dx1 += big_number / (1 + np.exp(X[0]/width))
    return [dx0, dx1]


plt.figure()

np.save(save_path+"/s.npy", s)
np.save(save_path+"/ts.npy", ts)
np.save(save_path+"/y.npy", xs)
for i in xrange(N_iterations):
    print 'iteration number ', i
    X=0
    with np.errstate(over='ignore'):
        X = odeint(f, [1, 0], ts, hmax=0.01)
    plt.plot(ts, X[:, 0], label=str(i))

    #Redefine M from these orbits
    M_interp = np.zeros(N_M)
    for j in xrange(N_M):
        x = xs[j]
        idx = (X[:,0]/Lambda(ts))<x
        M_interp[j] = 2.*s/3. * dt* (1./(ts[idx]**(1.+2.*s/3.))).sum()

    # Save everything
    np.save(save_path+"/l.npy", X)
    np.save(save_path+"/M.npy", M_interp)
    M_interp = interp1d(xs, M_interp, fill_value='extrapolate')

l = plt.legend()
plt.gca().set_xlabel(r'$\tau$')
plt.gca().set_ylabel(r'$\lambda$')
plt.show()

Interpolation routine to interpolate only between cusps, spline for optimal smoothness
-------------

In [ ]:
from scipy.interpolate import UnivariateSpline


N_M = int(1e4)
N_uni = 2


M_interp = np.zeros(N_M)
y_array = X[:,0]/Lambda(ts)


# Find reliable cusps
idx = ((y_array[1:-1] - y_array[:-2]) >0) & ((y_array[1:-1] - y_array[2:]) >0) & (y_array[1:-1]>1e-3)
y_cusp = y_array[1:-1][idx]


# Interpolation grid
y = np.linspace(1e-4, 1, N_M)
# Find the closest point next to a cusp, and make it exactly that point
for i in xrange(len(y_cusp)):
    y[ np.argmin(np.abs(y-y_cusp[i])) ] = y_cusp[i]

# Define mass profile:
for j in xrange(N_M):
    x = y[j]
    idx = y_array<x
    M_interp[j] = 2.*s/3. * dt* (1./(ts[idx]**(1.+2.*s/3.))).sum()

In [ ]:
# One spline per cusp.

M = np.zeros(N_M)
P = np.zeros(N_M)
sp = [None]*N_uni 
sp_d = [None]*N_uni

for i in xrange(N_uni):
    print 
    idx = ( y < y_cusp[i-1] ) & ( y >= y_cusp[i])
    if i==0:
        idx = ( y <= 1 ) & (y >= y_cusp[i])
        sp[i] = UnivariateSpline(y[idx], M_interp[idx])
    if i==(N_uni-1):
        idx = (y < y_cusp[i-1])
        sp[i] = UnivariateSpline(y[idx], M_interp[idx], k=5)
    else:
        sp[i] = UnivariateSpline(y[idx], M_interp[idx])
    #plt.plot(y, M_interp)
    sp_d[i] = sp[i].derivative()
    #plt.loglog(y[idx], sp_d[i](y[idx])/y[idx]/y[idx]/4/np.pi)
    M[idx] = sp[i](y[idx])
    P[idx] = sp_d[i](y[idx])/y[idx]/y[idx]/4/np.pi
    #plt.axvline(y_cusp[i])
    #if(i!=0) & (i!=N_uni):
    #    plt.axvline(y_cusp[i-1])

plt.plot(y, M_interp)
plt.plot(y, M)
plt.show()   
plt.loglog(y, P)
plt.show()

In [ ]:
#Smoooth density profile

def sigmoid(y):
    return np.exp(y*10.)/(np.exp(y*10.)+1.)

def connect(y, a, b, where):
    return a * sigmoid(    (y-y[where])/y[where]    ) + b * sigmoid( (y[where]-y)/y[where]   )

logy = np.log10(np.geomspace(1e-4, 1, int(1e4)))
logP = np.interp(np.log10(yp), np.log10(y), np.log10(P))


kernel = np.exp(-np.linspace(-50, 50, 20000-1)**2.)
kernel = kernel/kernel.sum()
logP_int = np.convolve(logP, kernel, 'valid')
logP_int2 = connect(logy, logP, logP_int, 6000)
logP_int = connect(logy, logP_int2, logP, -600)



P_int = 10.**np.interp(np.log10(y), logy, logP_int)
plt.figure(figsize=(10, 10))
plt.loglog(y, P_int)
plt.loglog(y, P)
plt.show()

In [ ]:
# Save smoothed version
np.save("cache_smooth/y.npy", y)
np.save("cache_smooth/M.npy", M)
np.save("cache_smooth/P.npy", P_int)
np.save("cache_smooth/ts.npy", ts)
np.save("cache_smooth/l.npy", X)
np.save("cache_smooth/s.npy", s)


# Save spiky version
np.save("cache/y.npy", y)
np.save("cache/M.npy", M)
np.save("cache/P.npy", P)
np.save("cache/ts.npy", ts)
np.save("cache/l.npy", X)
np.save("cache/s.npy", s)